In [71]:
import pandas as pd
import numpy as np
from matplotlib as plt
from pprint import pprint
import pickle
%matplotlib inline

In [2]:
# movie_data = pickle.load( open( "movie_data.pkl", "rb" ) )

maybe need to through out movies that are currently in theatres, because this data wont be accurate for those data points

# Parsing scraped data into Dataframe

In [403]:
movie_keys = ['title','domestic_total', 'runtime','rating', 'release_date' ]
weekly_keys = ['rank','num_theaters','weekly_gross', 'week_num']

In [404]:
def parse_value(num):
    try:
        num = int(num.replace('$', '').replace(',', ''))
    except:
        num = np.nan
    return num
    

In [405]:
def parse_weekly_data(weekly_dict):
    '''a function that creates the attributes that are a function 
       of the weekly data (e.g., drop_off_rate) and others. Minimum case yields
       the number of weeks in theatres.
    '''
    weeks = {}
    weekly_keys = ['rank','num_theaters','weekly_gross', 'week_num']
    for key in weekly_keys:
        weeks[key] = [parse_value(x) for x in weekly_dict[key]]
    return weeks

In [406]:
def parse_movie_data(movie_data):
    ''' Takes a dict of a movie scraped from Box Office Mojo and generates
        a row of for each feature to be added to the dataframe.
    '''
    mvp_movie_data = []
    for movie_dict in movie_data:
        # Only keep movies with correct initial values.
        if '1' == movie_dict['weekly_data']['week_num'][0]:
            movie = [movie_dict[key] for key in movie_keys]
            movie.append(max([int(x) for x in movie_dict['weekly_data']['week_num']]))
            movie.append(parse_weekly_data(movie_dict['weekly_data']))
            mvp_movie_data.append(movie) 
    return pd.DataFrame(mvp_movie_data)

In [407]:
df = parse_movie_data(movie_data)

Subsetted the data set to only include movies with data that started at week 1!

In [408]:
names = movie_keys + ['num_weeks','weekly_data']
df.columns = names

# Processing and transofrming the data for regression

Subset set df so that every movie ran for at least 2 weeks, otherwise, there would be nothing to predict for movies that ended in the first week. 

In [504]:
df = df[df['num_weeks']>3]

In [268]:
def get_features(df,current_week_num):
    ''' This function takes the main dataframe and a current week number and
        dynamically generates the full feature set by parsing the weekly data dict.
    '''
    new_data = []
    current_week_num = 1
    x = df[df['num_weeks']>current_week_num]
    for row in x.as_matrix():
        new_vals = [vals[current_week_num -1 ] for vals in row[6].values()]
        new_data.append(new_vals)
    d = pd.DataFrame(new_data)
    names = [s + str(current_week_num) for s in row[6].keys()]
    d.columns = names
    features = pd.concat([x, d], axis=1)
    return features

In [505]:
test = df[:10]

In [507]:
movie_weeks = pd.DataFrame(test.weekly_data[0].values()).T.as_matrix()

In [508]:
movie_weeks

array([[36022270,     3535,        2,        1],
       [20820708,     3544,        3,        2],
       [ 9247972,     2162,        7,        3],
       [ 6080223,     2153,        7,        4],
       [ 2575010,      796,       10,        5],
       [  702247,      428,       15,        6],
       [  453311,      296,       20,        7],
       [  432328,      282,       22,        8],
       [  312908,      241,       28,        9],
       [  225418,      204,       32,       10],
       [  157072,      151,       38,       11],
       [   90952,       92,       45,       12],
       [   75897,       54,       50,       13],
       [   25783,       31,       66,       14]])

In [495]:
names = test.weekly_data[0].keys()

In [496]:
test.head(2)

,title,domestic_total,runtime,rating,release_date,num_weeks,weekly_data
0,The A-Team,77222099,117,PG-13,2010-06-11,14,"{u'weekly_gross': [36022270, 20820708, 9247972..."
1,A.C.O.D.,175705,88,PG-13,2013-10-04,5,"{u'weekly_gross': [26439, 48909, 64916, 24656,..."


In [503]:
test

,0
title,The A-Team
domestic_total,77222099
runtime,117
rating,PG-13
release_date,2010-06-11 00:00:00
num_weeks,14
weekly_data,"{u'weekly_gross': [36022270, 20820708, 9247972..."
weekly_gross_1,36022270
num_theaters_1,3535
rank_1,2


In [498]:
for i in range(2):
    vals = movie_weeks[i]
    col_names = [name+'_%s'% str(i+1) for name in names]
    test = pd.concat([test.loc[i],pd.DataFrame(vals,col_names)])
    

KeyError: 'the label [1] is not in the [index]'

In [476]:
pd.concat([test.loc[0],pd.DataFrame(x[0],new_names)],axis= 0)

,0
title,The A-Team
domestic_total,77222099
runtime,117
rating,PG-13
release_date,2010-06-11 00:00:00
num_weeks,14
weekly_data,"{u'weekly_gross': [36022270, 20820708, 9247972..."
weekly_gross_1,36022270
num_theaters_1,3535
rank_1,2


,title,domestic_total,runtime,rating,release_date,num_weeks,weekly_data
0,The A-Team,77222099,117,PG-13,2010-06-11,14,"{u'weekly_gross': [36022270, 20820708, 9247972..."
1,A.C.O.D.,175705,88,PG-13,2013-10-04,5,"{u'weekly_gross': [26439, 48909, 64916, 24656,..."
2,A.I. Artificial Intelligence,78616689,145,PG-13,2001-06-29,11,"{u'weekly_gross': [45535716, 19346848, 7471014..."
4,Aarakshan,651096,NaN,Unrated,2011-08-12,6,"{u'weekly_gross': [440710, 159031, 39141, 1011..."
5,Aatsinki: The Story of Arctic Cowboys,6601,85,Unrated,2014-01-24,6,"{u'weekly_gross': [5359, 1242], u'num_theaters..."
6,Abandon,10727683,99,PG-13,2002-10-18,9,"{u'weekly_gross': [6023852, 2919275, 1149922, ..."
8,ABCD,222098,145,Unrated,2013-02-08,4,"{u'weekly_gross': [126154, 76711, 18176, 1057]..."
9,ABCD,892133,154,Unrated,2015-06-19,4,"{u'weekly_gross': [614396, 227713, 40239, 9785..."
10,ABCs of Death 2,7171,NaN,Unrated,2014-10-31,4,"{u'weekly_gross': [4944, 1661, 154, 412], u'nu..."
11,Abduction,28087155,106,PG-13,2011-09-23,12,"{u'weekly_gross': [13489188, 6979954, 3882349,..."


In [334]:
data = get_features(df,1)

# Need to build a smarter way to deal with missing values

In [335]:
test = data[['num_weeks','weekly_gross1','num_theaters1','rank1']]

In [342]:
test = test.dropna()

In [346]:
len(test)

4871

In [347]:
X = test[['weekly_gross1','num_theaters1','rank1']]

In [348]:
y = test['num_weeks']

In [349]:
with open('regression_data.pkl', 'wb') as output:
    pickle.dump((y,X), output, pickle.HIGHEST_PROTOCOL)